In [1]:
import os
import time
import numpy as np
from glob import glob
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import nrrd
import tifffile
import nibabel as nib
from medpy.io import load as medload
from medpy.io import save as medsave
from skimage.io import imread as skimread

In [2]:
f_tiff = 'sample_files/sample.tiff'
f_nrrd = 'sample_files/sample.nrrd'
f_nii = 'sample_files/sample.nii'

## save a random image

In [3]:
for i in range(1,11):
    fname = f'sample_files/sample_tiffs2d/sample{i}.tiff'
    img = np.random.randint(256, size=(32,16), dtype=np.uint8)
    medsave(img, fname)

In [10]:
img = np.random.randint(256, size=(2056,1024,1024), dtype=np.uint8)

In [11]:
tifffile.imwrite(f_tiff, img)

In [5]:
medsave(img, f_tiff)
medsave(img, f_nrrd)
medsave(img, f_nii)

## Compare loading times

In [ ]:
def time_func(func):
    def inner(*args, **kwargs):
        start_time = time.time()
        return_val = func(*args, **kwargs)
        print(time.time() - start_time)
        return return_val
    return inner

In [20]:
def option1(file_):
  if os.path.isdir(file_):
    fprefix = os.path.join(file_, '*.tif*')
    fnames = sorted(glob(fprefix))
 
    file_0 = tifffile.TiffFile(fnames[0]).asarray()
    if len(file_0.shape) == 2:
      img = np.stack([tifffile.imread(fname) for fname in fnames])
    else:
      img = np.concatenate([tifffile.imread(fname) for fname in fnames], dtype=file_0.dtype)
def option2(file_):
  if os.path.isdir(file_):
    fprefix = os.path.join(file_, '*.tif*')
    fnames = sorted(glob(fprefix))
 
    file_0 = tifffile.TiffFile(fnames[0]).asarray()
    #num_images = sum([len(tifffile.TiffFile(fname).pages) for fname in fnames])
    num_images=160
    #print(num_images)
    fshape = file_0.shape[1:] if len(file_0.shape)>2 else file_0.shape
    img = np.zeros((num_images, *fshape), dtype=file_0.dtype)
 
    if len(file_0.shape)>2:
      indx = 0
      for i,fname in enumerate(fnames):
        page = tifffile.imread(fname)
        img[indx : indx+len(page)] = page
        indx += len(page)
    else:
      for i,fname in enumerate(fnames):  
        img[i] = tifffile.imread(fname)

In [16]:
%timeit -r 100 -n 10 option1('sample_files/sample_tiffs3d')

13.3 ms ± 766 µs per loop (mean ± std. dev. of 100 runs, 10 loops each)


In [69]:
%timeit -r 100 -n 10 option2('sample_files/sample_tiffs3d')

13.4 ms ± 763 µs per loop (mean ± std. dev. of 100 runs, 10 loops each)


### tiff readers

In [60]:
%timeit -r 100 tifffile.imread(f_tiff)

1.44 s ± 14.1 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [61]:
%timeit -r 100 medload(f_tiff)

2.45 s ± 35.7 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [62]:
%timeit -r 100 skimread(f_tiff)

1.44 s ± 13 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


### nrrd readers

In [65]:
%timeit -r 100 nrrd.read(f_nrrd)

950 ms ± 3.86 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [66]:
%timeit -r 100 medload(f_nrrd)

2.53 s ± 30.7 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [67]:
%timeit -r 100 skimread(f_nrrd)

2.54 s ± 32.4 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


### nii readers

In [40]:
%timeit -r 100 nib.load(f_nii)

450 µs ± 5.39 µs per loop (mean ± std. dev. of 100 runs, 1000 loops each)


In [33]:
%timeit -r 100 np.array(nib.load(f_nii).dataobj)

893 ms ± 1.71 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [68]:
%timeit -r 100 medload(f_nii)

3.54 s ± 30.7 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [69]:
%timeit -r 100 skimread(f_nii)

3.54 s ± 28.8 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [41]:
%timeit -r 100 np.array(nib.load(f_nii).get_fdata())

10.1 s ± 64.3 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


## check reader

In [6]:
import univread

In [2]:
from importlib import reload
reload(univread)

<module 'univread' from '/home/ws/er5241/Repos/UnivRead3D/univread/__init__.py'>

In [3]:
img = univread.read('sample_files/sample_tiffs3d')
print(type(img))
print(img.shape)

<class 'numpy.ndarray'>
(160, 32, 64)


In [4]:
# check .nii.gz sample

img = univread.read('sample_files/sample.nii.gz')
print(type(img))
print(img.shape)

<class 'numpy.ndarray'>
(64, 32, 16)


In [5]:
# check unknown sample

img = univread.read('sample_files/sample.mha')
print(type(img))
print(img.shape)

<class 'numpy.ndarray'>
(64, 32, 16)


/home/ws/er5241/Repos/UnivRead3D/univread/univ_read.py:44: UserWarning: unrecognized file extension, proceeding to load with medpy
  warnings.warn('unrecognized file extension, proceeding to load with medpy')


## check mem-mapping

In [7]:
%load_ext memory_profiler

### tiff

In [23]:
%memit univread.read('sample_files/sample.tiff', lazy=False)

peak memory: 4222.41 MiB, increment: 1969.08 MiB


In [17]:
%memit univread.read('sample_files/sample.tiff', lazy=True)

peak memory: 2253.32 MiB, increment: 0.20 MiB


In [26]:
print(type(univread.read('sample_files/sample.tiff', lazy=False)))
print(type(univread.read('sample_files/sample.tiff', lazy=True)))

<class 'numpy.ndarray'>
<class 'numpy.memmap'>


### nii

In [24]:
%memit univread.read('sample_files/sample.nii', lazy=False)

peak memory: 5932.89 MiB, increment: 3679.56 MiB


In [25]:
%memit univread.read('sample_files/sample.nii', lazy=True)

peak memory: 2253.39 MiB, increment: 0.00 MiB


In [28]:
print(type(univread.read('sample_files/sample.nii', lazy=False)))
print(type(univread.read('sample_files/sample.nii', lazy=True)))

<class 'numpy.ndarray'>
<class 'nibabel.nifti1.Nifti1Image'>
